# 이모지 LoRA → 베이스 모델 머지

이모지 인식 LoRA 가중치를 베이스 모델에 합쳐 단일 모델로 만든다.  
사파리 에이전트 LoRA 학습 시 이모지 인식 능력이 내장된 베이스 위에서 학습하기 위함.

| 항목 | 값 |
|------|----|
| 베이스 모델 | `Qwen/Qwen3-VL-2B-Thinking` |
| LoRA 어댑터 | `adwel94/vision-emoji-recognition-lora` |
| 머지 결과 | `adwel94/Qwen3-VL-2B-Emoji-Base` |

CPU에서 실행 가능 (2B bf16 ≈ 4GB, RAM 16GB)

In [1]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
from peft import PeftModel
import torch

BASE_MODEL = "Qwen/Qwen3-VL-2B-Thinking"
LORA_ADAPTER = "adwel94/vision-emoji-recognition-lora"
MERGED_PATH = "./merged_model"

print("베이스 모델 로드...")
base_model = Qwen3VLForConditionalGeneration.from_pretrained(
    BASE_MODEL,
    dtype=torch.bfloat16,
    device_map="cpu",
    trust_remote_code=True,
)

print("LoRA 어댑터 로드...")
model = PeftModel.from_pretrained(base_model, LORA_ADAPTER)

print("프로세서 로드...")
processor = AutoProcessor.from_pretrained(BASE_MODEL, trust_remote_code=True)

print(f"베이스: {BASE_MODEL}")
print(f"LoRA: {LORA_ADAPTER}")
print(f"모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")

베이스 모델 로드...
LoRA 어댑터 로드...
프로세서 로드...


video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

C:\Users\hun41\miniconda3\envs\playground\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hun41\.cache\huggingface\hub\models--Qwen--Qwen3-VL-2B-Thinking. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


chat_template.json: 0.00B [00:00, ?B/s]

베이스: Qwen/Qwen3-VL-2B-Thinking
LoRA: adwel94/vision-emoji-recognition-lora
모델 파라미터: 2,144,964,608


In [2]:
print("어댑터 머지 중...")
merged_model = model.merge_and_unload()

print(f"머지 후 파라미터: {sum(p.numel() for p in merged_model.parameters()):,}")

print(f"로컬 저장: {MERGED_PATH}")
merged_model.save_pretrained(MERGED_PATH)
processor.save_pretrained(MERGED_PATH)

print("로컬 저장 완료")

어댑터 머지 중...
머지 후 파라미터: 2,127,532,032
로컬 저장: ./merged_model
로컬 저장 완료


In [3]:
from huggingface_hub import HfApi

HF_REPO = "adwel94/Qwen3-VL-2B-Emoji-Base"

api = HfApi()
api.create_repo(HF_REPO, exist_ok=True)

print(f"HF 업로드 시작: {HF_REPO}")
api.upload_folder(
    folder_path=MERGED_PATH,
    repo_id=HF_REPO,
    commit_message="Merge emoji recognition LoRA into base model",
)
print(f"업로드 완료: https://huggingface.co/{HF_REPO}")

HF 업로드 시작: adwel94/Qwen3-VL-2B-Emoji-Base


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...otebook\merged_model\tokenizer.json:  73%|#######2  | 8.30MB / 11.4MB            

  ...book\merged_model\model.safetensors:   1%|          | 41.9MB / 4.26GB            

업로드 완료: https://huggingface.co/adwel94/Qwen3-VL-2B-Emoji-Base


In [4]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
import torch

HF_REPO = "adwel94/Qwen3-VL-2B-Emoji-Base"

print(f"검증: {HF_REPO} 로드...")
verify_model = Qwen3VLForConditionalGeneration.from_pretrained(
    HF_REPO,
    dtype=torch.bfloat16,
    device_map="cpu",
    trust_remote_code=True,
)
verify_processor = AutoProcessor.from_pretrained(HF_REPO, trust_remote_code=True)

print(f"파라미터: {sum(p.numel() for p in verify_model.parameters()):,}")
print("검증 완료 — 머지 모델 정상 로드")

검증: adwel94/Qwen3-VL-2B-Emoji-Base 로드...


config.json: 0.00B [00:00, ?B/s]

C:\Users\hun41\miniconda3\envs\playground\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hun41\.cache\huggingface\hub\models--adwel94--Qwen3-VL-2B-Emoji-Base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/858 [00:00<?, ?B/s]

파라미터: 2,127,532,032
검증 완료 — 머지 모델 정상 로드
